## Import the required libraries

In [16]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import cv2
from PIL import Image
import os 
import time

## Load and Process the data

In [17]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 150
width = 150
channels = 3
classes = 43
n_inputs = height * width * channels

for i in range(0, classes, 1):
    path = "./TrafficSigns_dataset/train/{0}/".format(i)
    print("Loading Class {}...".format(i)+path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            size_image = cv2.resize(image, (height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print("Attribut Error")
            
Cells=np.array(data)
labels=np.array(labels)

Loading Class 0..../TrafficSigns_dataset/train/0/
Loading Class 1..../TrafficSigns_dataset/train/1/
Loading Class 2..../TrafficSigns_dataset/train/2/
Loading Class 3..../TrafficSigns_dataset/train/3/
Loading Class 4..../TrafficSigns_dataset/train/4/
Loading Class 5..../TrafficSigns_dataset/train/5/
Loading Class 6..../TrafficSigns_dataset/train/6/
Loading Class 7..../TrafficSigns_dataset/train/7/
Loading Class 8..../TrafficSigns_dataset/train/8/
Loading Class 9..../TrafficSigns_dataset/train/9/
Loading Class 10..../TrafficSigns_dataset/train/10/
Loading Class 11..../TrafficSigns_dataset/train/11/
Loading Class 12..../TrafficSigns_dataset/train/12/
Loading Class 13..../TrafficSigns_dataset/train/13/
Loading Class 14..../TrafficSigns_dataset/train/14/
Loading Class 15..../TrafficSigns_dataset/train/15/
Loading Class 16..../TrafficSigns_dataset/train/16/
Loading Class 17..../TrafficSigns_dataset/train/17/
Loading Class 18..../TrafficSigns_dataset/train/18/
Loading Class 19..../TrafficSign

In [18]:
#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(classes)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [19]:
#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

In [20]:
#Using one hote encoding for the train and validation labels
y_train = tf.keras.utils.to_categorical(y_train, classes)
y_val = tf.keras.utils.to_categorical(y_val, classes)

print(X_train.shape)

(31368, 15, 15, 3)


## Build and Compile the model

In [21]:
#Build the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(rate=0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 11, 11, 16)        1216      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 32)          4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [22]:
#Compilation of the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

## Train the model

In [23]:
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 10
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))

Epoch 1/10
981/981 [==============================] - 2s 2ms/step - loss: 1.5779 - accuracy: 0.5582 - val_loss: 0.5654 - val_accuracy: 0.8436
Epoch 2/10
981/981 [==============================] - 2s 2ms/step - loss: 0.6019 - accuracy: 0.8132 - val_loss: 0.2938 - val_accuracy: 0.9181
Epoch 3/10
981/981 [==============================] - 2s 2ms/step - loss: 0.4127 - accuracy: 0.8721 - val_loss: 0.2096 - val_accuracy: 0.9453
Epoch 4/10
981/981 [==============================] - 2s 2ms/step - loss: 0.3295 - accuracy: 0.8967 - val_loss: 0.1758 - val_accuracy: 0.9540
Epoch 5/10
981/981 [==============================] - 2s 2ms/step - loss: 0.2698 - accuracy: 0.9135 - val_loss: 0.1420 - val_accuracy: 0.9617
Epoch 6/10
981/981 [==============================] - 2s 2ms/step - loss: 0.2348 - accuracy: 0.9245 - val_loss: 0.1262 - val_accuracy: 0.9651
Epoch 7/10
981/981 [==============================] - 2s 2ms/step - loss: 0.2106 - accuracy: 0.9327 - val_loss: 0.1127 - val_accuracy: 0.9674
Epoch 

## Evaluate the model on test data

In [24]:
#Load test data
y_test=pd.read_csv("./TrafficSigns_dataset/Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values

print (y_test)

data=[]

for f in labels:
    image=cv2.imread('./TrafficSigns_dataset/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 

[16  1 38 ...  6  7 10]


In [25]:
#Predict output for test examples
pred = model.predict_classes(X_test)

In [26]:
#Calculate accuracy
correct = 0
incorrect = 0
if(len(pred) == len(y_test)):
    for i in range(0, len(pred)):
        if (pred[i] == y_test[i]):
            correct+=1
        else:
           incorrect+=1

test_acc = correct/(correct+incorrect)
print('Test accuracy:', test_acc)

Test accuracy: 0.8975455265241489


## Save the model

In [27]:
model.save("BlackBox_CNN.h5")